In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [18]:
from pydantic import BaseModel
from typing import Literal

class llm_schema(BaseModel):
    movie_summary_flag : Literal["positive", "negative"]

llm_structured_output = llm_gemini.with_structured_output(llm_schema)

result = llm_structured_output.invoke("One of the best movie")
result.model_dump()['movie_summary_flag']

'positive'

# **CHAIN With Conditional Chains**

In [19]:
# Task-1 [prompt] 
prompt_template = ChatPromptTemplate.from_messages(
    ["system","You are movie review evaluator.",
    ("human","Please categorize the movie review as positive or negative:{input}")
    ])


In [28]:

# Task-2 [LLM]
llm_structured_output = llm_gemini.with_structured_output(llm_schema)

# Task 3 [custom runnable]
def pydantic_json(input: llm_schema)->str:
    return input.model_dump_json()

pydantic_json_lambda = RunnableLambda(pydantic_json)


In [21]:

# Task 4 [Custom runnable]
from langchain_core.runnables import RunnableLambda,RunnableBranch

def dictionary_maker(text:str)->dict:
    return {"text":text}

dictionary_maker_runnable = RunnableLambda(dictionary_maker)    

# **Conditional Chain 1**

In [22]:
from langchain_core.output_parsers import StrOutputParser
# Task 1 [Prompt]
linkdin_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a linkdin post generator "),
    ("human","create a post for the following text for linkdin : {text}")
])

# Task 2 [LLM]
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-09-2025",temperature=0)

# Task 3 [Str Parser]
str_parser = StrOutputParser()

chain_linkdin = linkdin_prompt | llm_gemini | str_parser

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


# **Conditional Chain 2**

In [23]:
from langchain_core.runnables import RunnableParallel,RunnableLambda,RunnableBranch

In [24]:
def insta_chain(text:dict):
    text = text["text"]

    # Task 1 [Prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
        ("system","You are a insta post generator "),
        ("human","create a post for the following text for insta : {text}")
    ])

    # Task 2 [LLM]
    llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-09-2025",temperature=0)

    # Task 3 [Str Parser]
    str_parser = StrOutputParser()

    chain_insta = insta_prompt | llm_gemini | str_parser

    result = chain_insta.invoke(text)
    return result   


insta_chain_runnable = RunnableLambda(insta_chain)


# **Final Orchestration**

In [29]:
conditional_chain = RunnableBranch(
   (lambda x: "positive" in x , chain_linkdin),
   insta_chain_runnable
    )

final_orchestrator =  prompt_template | llm_structured_output | pydantic_json_lambda | conditional_chain

In [30]:
final_orchestrator.invoke({"input":"I love this chhava movie"})

'Here are a few options for a LinkedIn post based on the input `{"movie_summary_flag":"positive"}`, ranging in tone and focus. Choose the one that best fits your personal brand!\n\n---\n\n## Option 1: Enthusiastic & Engaging (Focus on Inspiration)\n\n**Post:**\n\nJust finished watching a movie that genuinely left me feeling uplifted and inspired! 🤩\n\nIt’s amazing how powerful storytelling can be in shifting perspective and injecting a dose of positivity into your week. Sometimes, all you need is a great narrative to recharge your batteries.\n\nWhat\'s the last film or show that left you feeling genuinely positive? Drop your recommendations below! 👇\n\n#MovieNight #Inspiration #PositiveVibes #Storytelling #Entertainment\n\n---\n\n## Option 2: Professional & Reflective (Focus on Quality/Impact)\n\n**Post:**\n\nIt’s rare to find content that delivers such a high level of quality *and* leaves you with a distinctly positive takeaway.\n\nI recently experienced a film that truly nailed the b

# *Chain as runnable*

In [13]:
# task 1 Butify Function

def butify(final_response :dict)->dict:
    linkedin_response = final_response["branches"]["linkedin"]
    instagram_response = final_response["branches"]["instagram"]
    return {"linkedin":linkedin_response,"instagram":instagram_response}

beautify_runnable = RunnableLambda(butify)

# Task 2 Final Chain - Beautified chain
beautified_chain = final_chain | beautify_runnable

beautified_chain.invoke("pushpa")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


{'linkedin': 'Here are a few options for a LinkedIn post based on the text, catering to different angles:\n\n---\n\n## Option 1: Focusing on Leadership & Ambition (More Professional Tone)\n\n**🎬 From Coolie to Kingpin: Lessons in Unstoppable Ambition from \'Pushpa: The Rise\'**\n\nThe story of **Pushpa Raj** in *Pushpa: The Rise* is more than just an action film; it\'s a masterclass in strategic ascent within a complex, high-stakes environment.\n\nWatching Pushpa navigate the brutal hierarchy of the red sandalwood smuggling syndicate in the Seshachalam forests offers fascinating parallels to corporate strategy:\n\n* **Identifying the Gap:** Recognizing an opportunity where others see only risk.\n* **Ruthless Execution:** The importance of decisive action when climbing the ladder.\n* **Building an Empire:** Understanding the ecosystem you operate within, whether it\'s a forest syndicate or a market sector.\n\nIt’s a gritty reminder that true leadership often requires navigating the grey